In [6]:
import mlflow
import mlflow.pytorch
import json
import torch
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn import CrossEntropyLoss
from torch_geometric.loader import DataLoader
from torch_geometric.nn.models import GAT
import os
from torch_geometric.data import Data
from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
from torch_geometric.data.storage import GlobalStorage
import torch.serialization
import optuna
from sklearn.model_selection import KFold
import numpy as np

# from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# import numpy as np




In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:

# Load data 
with torch.serialization.safe_globals([Data]):
    data_list = torch.load("training_data\Datacheckpoint_latest_22", map_location='cuda', weights_only=False)

labels = json.load(open("label_encoding.json"))
batch_size = 1

train_split = int(len(data_list) * 0.8)
train_data = data_list[:train_split]
val_data = data_list[train_split:]

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

in_channels = data_list[0].x.size(1)
# in_channels =18
num_classes = len(labels)

model_dir = "C:\\Users\\User\\OneDrive\\Desktop\\GAT-model testing\\GAT-test\\models\\model"
results_dir = "C:\\Users\\User\\OneDrive\\Desktop\\GAT-model testing\\GAT-test\\results"
plots_dir = "C:\\Users\\User\\OneDrive\\Desktop\\GAT-model testing\\GAT-test\\plots"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [9]:
from sklearn.model_selection import KFold
import mlflow
import mlflow.pytorch




def objective(trial):
    config = {
        'hidden_channels': trial.suggest_categorical('hidden_channels', [64, 128, 256]),
        'num_layers': trial.suggest_int('num_layers', 1, 3),
        'heads': trial.suggest_categorical('heads', [1, 2, 4, 8]),
        'dropout': trial.suggest_float('dropout', 0.0, 0.5),
        'hidden_dim': trial.suggest_categorical('hidden_dim', [64, 128, 256])
    }

    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        mlflow.log_params(config)
        mlflow.set_tag("cv_strategy", "KFold")
        mlflow.set_tag("model", "GATv2")

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        data_indices = list(range(len(data_list)))
        fold_val_acc = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(data_indices)):
            train_data = [data_list[i] for i in train_idx]
            val_data = [data_list[i] for i in val_idx]

            train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_data, batch_size=batch_size)

            model = GAT(
                in_channels=in_channels,
                hidden_channels=config['hidden_channels'],
                num_layers=config['num_layers'],
                out_channels=num_classes,
                dropout=config['dropout'],
                heads=config['heads'],
                v2=True,
                edge_dim=1,
                jk='lstm'
            ).to(device)

            all_train_labels = torch.cat([data.y for data in train_loader.dataset])
            class_weights = 1.0 / (torch.bincount(all_train_labels, minlength=num_classes).float() + 1e-6)
            class_weights = class_weights / class_weights.sum()
            class_weights = class_weights.to(device)

            criterion = CrossEntropyLoss(weight=class_weights)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=5e-4)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

            best_val_acc = 0
            best_val_loss = float('inf')
            wait = 0
            patience = 30
            min_delta = 1e-4

            training_loss, validation_loss, validation_acc = [], [], []

            for epoch in range(501):
                model.train()
                total_loss = 0
                correct_train = 0
                total_train = 0

                for data in train_loader:
                    data = data.to(device)
                    optimizer.zero_grad()
                    out = model(data.x, data.edge_index, edge_weight=data.edge_attr)
                    loss = criterion(out, data.y)
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                    correct_train += (out.argmax(dim=1) == data.y).sum().item()
                    total_train += data.y.size(0)

                avg_train_loss = total_loss / len(train_loader)
                train_acc = correct_train / total_train
                training_loss.append(avg_train_loss)

                model.eval()
                val_loss = 0
                correct, total = 0, 0
                with torch.no_grad():
                    for data in val_loader:
                        data = data.to(device)
                        out = model(data.x, data.edge_index, edge_weight=data.edge_attr)
                        loss = criterion(out, data.y)
                        val_loss += loss.item()
                        correct += (out.argmax(dim=1) == data.y).sum().item()
                        total += data.y.size(0)

                avg_val_loss = val_loss / len(val_loader)
                val_acc = correct / total
                validation_loss.append(avg_val_loss)
                validation_acc.append(val_acc)

                # Log metrics per epoch
                mlflow.log_metric(f"fold{fold+1}_train_loss", avg_train_loss, step=epoch)
                mlflow.log_metric(f"fold{fold+1}_val_loss", avg_val_loss, step=epoch)
                mlflow.log_metric(f"fold{fold+1}_val_acc", val_acc, step=epoch)
                mlflow.log_metric(f"fold{fold+1}_val_acc", train_acc, step=epoch)

                scheduler.step(avg_val_loss)
                trial.report(val_acc, epoch)

                print(f"Epoch {epoch+1:03d} | Fold {fold+1} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Train loss: {avg_train_loss:.4f} Val Loss: {avg_val_loss:.4f}")

                if avg_val_loss < best_val_loss - min_delta:
                    best_val_loss = avg_val_loss
                    best_val_acc = val_acc
                    wait = 0
                else:
                    wait += 1
                    if wait >= patience:
                        print(f" Early stopping at epoch {epoch+1} for fold {fold+1}")
                        break

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

            print(f" Fold {fold+1}: Best Val Acc = {best_val_acc:.4f}")
            fold_val_acc.append(float(best_val_acc))

        mean_val_acc = float(np.mean(fold_val_acc))
        mlflow.log_metric("mean_val_acc", mean_val_acc)

        return mean_val_acc


In [10]:
import mlflow

# Set MLflow experiment name — all trials/logs will be grouped under this
mlflow.set_experiment("GATv2_Optuna_Experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000") 

# Create and run the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best trial:")
print("Accuracy:", study.best_trial.value)
print("Params:", study.best_trial.params)


2025/07/15 00:08:40 INFO mlflow.tracking.fluent: Experiment with name 'GATv2_Optuna_Experiment' does not exist. Creating a new experiment.
[I 2025-07-15 00:08:40,326] A new study created in memory with name: no-name-d7f9bc4c-8f1c-4e3f-b2d0-d43f3f87b3a4


Epoch 001 | Fold 1 | Train Acc: 0.1868 | Val Acc: 0.4528 | Train loss: 1.3807 Val Loss: 1.3555
Epoch 002 | Fold 1 | Train Acc: 0.2979 | Val Acc: 0.4686 | Train loss: 1.3652 Val Loss: 1.3473
Epoch 003 | Fold 1 | Train Acc: 0.4733 | Val Acc: 0.5157 | Train loss: 1.3488 Val Loss: 1.3248
Epoch 004 | Fold 1 | Train Acc: 0.5548 | Val Acc: 0.5535 | Train loss: 1.3283 Val Loss: 1.2932
Epoch 005 | Fold 1 | Train Acc: 0.5089 | Val Acc: 0.5440 | Train loss: 1.3017 Val Loss: 1.2371
Epoch 006 | Fold 1 | Train Acc: 0.4076 | Val Acc: 0.6541 | Train loss: 1.2631 Val Loss: 1.1631
Epoch 007 | Fold 1 | Train Acc: 0.4960 | Val Acc: 0.6792 | Train loss: 1.1974 Val Loss: 1.0602
Epoch 008 | Fold 1 | Train Acc: 0.5504 | Val Acc: 0.6792 | Train loss: 1.0936 Val Loss: 0.8141
Epoch 009 | Fold 1 | Train Acc: 0.5227 | Val Acc: 0.7138 | Train loss: 0.9169 Val Loss: 0.5829
Epoch 010 | Fold 1 | Train Acc: 0.4960 | Val Acc: 0.7421 | Train loss: 0.7799 Val Loss: 0.4817
Epoch 011 | Fold 1 | Train Acc: 0.5845 | Val Acc: 

c:\Users\User\anaconda3\envs\ENVGAT\lib\site-packages\optuna\trial\_trial.py:501: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


Epoch 001 | Fold 2 | Train Acc: 0.2032 | Val Acc: 0.2212 | Train loss: 1.3861 Val Loss: 1.3747


c:\Users\User\anaconda3\envs\ENVGAT\lib\site-packages\optuna\trial\_trial.py:501: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


...

Epoch 024 | Fold 1 | Train Acc: 0.5306 | Val Acc: 0.7516 | Train loss: 0.6560 Val Loss: 0.4648
🏃 View run trial_27 at: http://127.0.0.1:5000/#/experiments/906754211394743286/runs/3df79352a3da4eeda45b204e8eefa738
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/906754211394743286
Epoch 001 | Fold 1 | Train Acc: 0.5242 | Val Acc: 0.3648 | Train loss: 1.3874 Val Loss: 1.3875


[I 2025-07-15 01:12:45,305] Trial 28 pruned. 


Epoch 002 | Fold 1 | Train Acc: 0.5109 | Val Acc: 0.3522 | Train loss: 1.3689 Val Loss: 1.3553
🏃 View run trial_28 at: http://127.0.0.1:5000/#/experiments/906754211394743286/runs/7fd567ae0dce4f02ac2f3e98fe0c7a6a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/906754211394743286
Epoch 001 | Fold 1 | Train Acc: 0.1986 | Val Acc: 0.4560 | Train loss: 1.3859 Val Loss: 1.3622
Epoch 002 | Fold 1 | Train Acc: 0.2737 | Val Acc: 0.5409 | Train loss: 1.3722 Val Loss: 1.3461
Epoch 003 | Fold 1 | Train Acc: 0.3839 | Val Acc: 0.6572 | Train loss: 1.3568 Val Loss: 1.3316
Epoch 004 | Fold 1 | Train Acc: 0.3577 | Val Acc: 0.6384 | Train loss: 1.3390 Val Loss: 1.2979
Epoch 005 | Fold 1 | Train Acc: 0.4580 | Val Acc: 0.6761 | Train loss: 1.3188 Val Loss: 1.2757
Epoch 006 | Fold 1 | Train Acc: 0.5524 | Val Acc: 0.6761 | Train loss: 1.2833 Val Loss: 1.2213
Epoch 007 | Fold 1 | Train Acc: 0.4946 | Val Acc: 0.6887 | Train loss: 1.2342 Val Loss: 1.1198


[I 2025-07-15 01:12:50,179] Trial 29 pruned. 


Epoch 008 | Fold 1 | Train Acc: 0.4580 | Val Acc: 0.6667 | Train loss: 1.1481 Val Loss: 0.9260
🏃 View run trial_29 at: http://127.0.0.1:5000/#/experiments/906754211394743286/runs/c9053c68dbd54fba8b1d06b3e80e47ba
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/906754211394743286
Best trial:
Accuracy: 0.717528137604627
Params: {'hidden_channels': 256, 'num_layers': 2, 'heads': 1, 'dropout': 0.09613910035252272, 'hidden_dim': 256}


In [13]:
print("Best trial:")
print("Accuracy:", study.best_trial.value)
print("Params:", study.best_trial.params)

Best trial:
Accuracy: 0.717528137604627
Params: {'hidden_channels': 256, 'num_layers': 2, 'heads': 1, 'dropout': 0.09613910035252272, 'hidden_dim': 256}


In [12]:
data_list[0].y

tensor([0, 1, 3, 2, 0, 3, 3, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 3, 1, 2, 2, 0, 2,
        3, 2, 1, 0, 1, 3, 3, 2, 0, 2, 2, 1, 0, 1, 2, 2, 0, 3, 2, 2, 1, 0, 1, 2,
        2, 0, 1, 2, 2, 0, 2, 2, 1, 0, 1, 2, 3, 0, 2, 1, 2, 2, 0, 1, 3, 2, 0, 3,
        3, 3, 1, 0, 2, 2, 1, 0, 2, 2, 1, 0, 1, 2, 2, 3, 3, 3, 3, 0, 2, 2, 2, 1,
        0, 1, 2, 0, 2, 2, 1], device='cuda:0')

In [ ]:
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
from torch.nn import CrossEntropyLoss
from torch_geometric.loader import DataLoader

# ----------------- SET MLFLOW CONFIG ----------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("GATv2_Final_Training")

# ----------------- LOAD BEST HYPERPARAMETERS ----------------------
best_params_path = os.path.join(model_dir, "best_params.json")
best_params = study.best_trial.params
with open(best_params_path, "w") as f:
    json.dump(best_params, f, indent=4)

# ----------------- SETUP FINAL TRAINING ----------------------
final_model = GAT(
    in_channels=in_channels,
    hidden_channels=best_params['hidden_channels'],
    num_layers=best_params['num_layers'],
    out_channels=num_classes,
    dropout=best_params['dropout'],
    heads=best_params['heads'],
    v2=True,
    edge_dim=1,
    jk='lstm'
).to(device)

# Class-weighted loss
all_labels = torch.cat([data.y for data in train_loader.dataset])
class_counts = torch.bincount(all_labels, minlength=num_classes)
class_weights = 1.0 / (class_counts.float() + 1e-6)
class_weights = class_weights / class_weights.sum()
class_weights = class_weights.to(device)

criterion = CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(final_model.parameters(), lr=0.0005, weight_decay=5e-4)

train_losses = []
val_losses = []
best_val_loss = float('inf')
best_model_path = os.path.join(model_dir, "GAT_full_model_best_001.pt")

# ----------------- TRAINING LOOP ----------------------
for epoch in range(500):
    final_model.train()
    train_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = final_model(data.x, data.edge_index, edge_weight=data.edge_attr)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    final_model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            out = final_model(data.x, data.edge_index, edge_weight=data.edge_attr)
            loss = criterion(out, data.y)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    # Save best weights
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(final_model.state_dict(), best_model_path)

    print(f"[FINAL TRAIN] Epoch {epoch+1:03d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# Save full model (.pt)
full_model_path = os.path.join(model_dir, "GAT_final_model.pt")
torch.save(final_model, full_model_path)
print(f"Full model saved to {full_model_path}")
print(f" Best model (lowest val loss) saved to {best_model_path}")

# ----------------- PLOT LOSS CURVE ----------------------
plt.figure(figsize=(8,5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Final Model Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plot_path = os.path.join(plots_dir, "final_train_val_loss.png")
plt.savefig(plot_path)
plt.show()

# ----------------- MLFLOW LOGGING ----------------------
# Load best weights before logging
final_model.load_state_dict(torch.load(best_model_path))

with mlflow.start_run(run_name="final_retrain"):
    mlflow.log_params(best_params)
    mlflow.log_artifact(best_params_path)
    mlflow.log_artifact(plot_path)

    for epoch, (train_l, val_l) in enumerate(zip(train_losses, val_losses)):
        mlflow.log_metric("train_loss", train_l, step=epoch)
        mlflow.log_metric("val_loss", val_l, step=epoch)

    # ✅ Log model
    mlflow.pytorch.log_model(final_model, artifact_path="final_model")
    print(" Final model logged to MLflow.")


In [34]:
print("Type:", type(data_list[0].y))
print("Shape:", data_list[0].y.shape)
print("Value:", data_list[0].y)


Type: <class 'torch.Tensor'>
Shape: torch.Size([103])
Value: tensor([0, 1, 3, 2, 0, 3, 3, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 3, 1, 2, 2, 0, 2,
        3, 2, 1, 0, 1, 3, 3, 2, 0, 2, 2, 1, 0, 1, 2, 2, 0, 3, 2, 2, 1, 0, 1, 2,
        2, 0, 1, 2, 2, 0, 2, 2, 1, 0, 1, 2, 3, 0, 2, 1, 2, 2, 0, 1, 3, 2, 0, 3,
        3, 3, 1, 0, 2, 2, 1, 0, 2, 2, 1, 0, 1, 2, 2, 3, 3, 3, 3, 0, 2, 2, 2, 1,
        0, 1, 2, 0, 2, 2, 1], device='cuda:0')
